# Example: Divergence method

A brief example demonstrating the application of the divergence method to estimate CO2 and NO2 emissions of the Boxberg power plant in the SMARTCARB dataset. The example requires both the SMARTCARB Level-2 data and the wind fields that you be downloaded here: https://doi.org/10.5281/zenodo.4048227

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
import numpy as np
import ddeq

In [ ]:
ROOT = '/project/coco2/fileshare/WP4/SMARTCARB/'
SMARTCARB_DATA_PATH = os.path.join(ROOT, 'level2')
SMARTCARB_WIND_PATH = os.path.join(ROOT, 'wind_fields')

In [ ]:
sources = ddeq.misc.read_point_sources()
sources = sources.sel(source=['Boxberg'])

l2 = ddeq.smartcarb.Level2Dataset(SMARTCARB_DATA_PATH, 'ace', co2_noise_scenario='low', no2_noise_scenario='low',
                                  make_no2_error_cloud_dependent=False)

res = ddeq.div.estimate_emissions(
    l2, SMARTCARB_WIND_PATH, sources,
    wind_product='SMARTCARB', trace_gases=['CO2', 'NO2'],
    start_date='2015-02-01', end_date='2015-05-01')

In [ ]:
res

The estimated emissions of Boxberg are 760.2$\pm$61.56 kg/s and 0.5766$\pm$0.06649 kg/s for CO2 and NO2 using low-noise CO2 and NO2 for constellation ace from 1 Feb to 1 May 2015.

## Compare with true emissions

In [ ]:
times = pd.date_range('2015-02-01', '2015-05-01')
co2_true = np.mean([ddeq.smartcarb.read_true_emissions(t, 'CO2', 'Boxberg').mean() for t in times])
no2_true = np.mean([ddeq.smartcarb.read_true_emissions(t, 'NO2', 'Boxberg').mean() for t in times])

In [ ]:
co2_est = ddeq.misc.kgs_to_Mtyr(res.sel(source='Boxberg')['CO2_estimated_emissions'])
co2_pre = ddeq.misc.kgs_to_Mtyr(res.sel(source='Boxberg')['CO2_estimated_emissions_precision'])

no2_est = 1e3 * ddeq.misc.kgs_to_Mtyr(res.sel(source='Boxberg')['NO2_estimated_emissions'])
no2_pre = 1e3 * ddeq.misc.kgs_to_Mtyr(res.sel(source='Boxberg')['NO2_estimated_emissions_precision'])

print(f'CO2 estimate: {co2_est:.1f}±{co2_pre:.1f} Mt/a, True: {co2_true:.1f} Mt/a')
print(f'NO2 estimate: {no2_est:.1f}±{no2_pre:.1f} kt/a, True: {no2_true:.1f} kt/a')